In [ ]:
%run ./00_Mapping_Setup

In [ ]:
/* ===================================================================================
   METRIC: EMP03 - Code of Conduct Timely Training Completion
   
   BUSINESS QUESTION: 
   Percentage of the unit's staff that have timely completed and acknowledged...
   
   LOGIC SUMMARY:
   1. Evaluates each employee's Training Record from the td_code_of_conduct table.
   2. Logic: Completion date must be <= Due Date. If due date is blank, mark as 
      non-completed.
   3. Joins the training data to the standardized CC Mapping Bootstrap view using 
      INT casting to prevent leading-zero mismatch.
   4. Rolls up the data by AU_ID (BusinessID) to calculate the percentage:
      (Completed On Time / Total Employees) * 100
   5. Formats the output to match the final reporting template columns.
=================================================================================== */

WITH Training_Data AS (
    -- 1. Evaluate each employee to see if they completed the training on time
    SELECT 
        -- Cast to INT to strip leading zeros for a perfect join
        CAST(TRIM(DeptID) AS INT) AS Cleaned_DeptID,
        
        -- Logic: 1 = On Time, 0 = Not On Time / Incomplete
        CASE 
            -- ERA Note: "If due date is blank, mark as non-completed"
            WHEN DueDate IS NULL OR TRIM(DueDate) = '' THEN 0
            
            -- If CompletionDate is blank, it's non-completed
            WHEN CompletionDateUTC IS NULL OR TRIM(CompletionDateUTC) = '' THEN 0
            
            -- "Completion date before Due Date"
            WHEN CAST(CompletionDateUTC AS DATE) <= CAST(DueDate AS DATE) THEN 1
            
            -- Missed the deadline
            ELSE 0 
        END AS Completed_On_Time
        
    FROM hive_metastore.ra_adido_2025.td_code_of_conduct_and_ethics_11012024_10312025
    WHERE DeptID IS NOT NULL
),

Mapped_AUs AS (
    -- 2. Join the base AU list to the Training matches
    SELECT 
        m.AU_ID AS BusinessID,
        t.Completed_On_Time,
        -- Flag if an employee exists so we have an accurate denominator
        CASE WHEN t.Cleaned_DeptID IS NOT NULL THEN 1 ELSE 0 END AS Has_Employee
    FROM vw_cost_center_mapping_bootstrap m
    
    -- LEFT JOIN ensures ALL Assessable Units show up in the final template
    LEFT JOIN Training_Data t
        ON CAST(m.Cost_Center_ID AS INT) = t.Cleaned_DeptID
)

-- 3. Roll everything up to match the Final Output Template
SELECT 
    BusinessID,                          -- Maps to Column A
    'EMP03' AS QuestionID,               -- Maps to Column B
    'Applicable' AS Applicability,       -- Maps to Column C
    '' AS ApplicabilityRationale,        -- Maps to Column D
    
    -- Maps to Column E: Calculates the percentage and formats it neatly
    CASE 
        -- Fallback: If an Assessable Unit has 0 mapped employees in the training file
        WHEN SUM(Has_Employee) = 0 THEN '0%' 
        ELSE CONCAT(
            CAST(ROUND((SUM(Completed_On_Time) * 100.0) / SUM(Has_Employee), 2) AS STRING), 
            '%'
        )
    END AS Response
    
FROM Mapped_AUs
GROUP BY BusinessID
ORDER BY BusinessID;